In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")
sys.path.append("../")
import os
print(os.getcwd())
from models.basic import CNN
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# width_to_output_mapping = {}
# device = "cuda"

# cnn = CNN(nc=1, cnn_type="default64", first_conv_opts=None)
# cnn.to(device)
# for width in range(233,240):
#     try:
#         t = torch.zeros(1, 1, 32, width).to(device)
#     except:
#         device = "cpu"
#         t = torch.zeros(1, 1, 32, width).to(device)

#     shape = cnn(t).shape
#     width_to_output_mapping[width] = shape[0]
    

/media/data/GitHub/simple_hwr/hwr_utils/testing


In [2]:
import tempfile
from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
from models.basic import CNN, BidirectionalRNN
from torch import nn
from loss_module.stroke_recovery_loss import StrokeLoss
import torch
from models.CoordConv import CoordConv
from trainers import TrainerStrokeRecovery
from hwr_utils.stroke_dataset import StrokeRecoveryDataset, read_img
from hwr_utils.stroke_recovery import *
from hwr_utils import utils, stroke_recovery
from torch.optim import lr_scheduler
from timeit import default_timer as timer
from train_stroke_recovery import graph
from hwr_utils.distortions import *
from hwr_utils.stroke_plotting import *

## DRAW THE IMAGE
#Pillow-simd

from PIL import Image, ImageDraw

def loader():
    folder = Path("online_coordinate_data/3_stroke_vSmall")
    folder = Path("online_coordinate_data/8_stroke_vSmall_16")
    folder = Path("online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull")

    print(os.getcwd())
    x_relative_positions = True
    test_size = 500
    train_size = None
    batch_size=16
    #gt_format = ["x","y","stroke_number","eos"]
    gt_format = ["x","y","sos","eos"]

    test_dataset=StrokeRecoveryDataset([folder / "test_online_coords.json"],
                            img_height = 61,
                            num_of_channels = 1.,
                            max_images_to_load = test_size,
                            root=r"../../data",
                            cnn=None,
                            gt_format=gt_format,
                            )

    test_dataloader = DataLoader(test_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=3,
                                  collate_fn=test_dataset.collate,
                                  pin_memory=False)

    device="cuda"
    globals().update(locals())
    return test_dataloader

def reload_and_get_new_example():
    test_dataloader = loader()
    return next(iter(test_dataloader))

def get_instance():
#     if not 'test_dataloader' in globals():
#         test_dataloader = loader()
    example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
    instance = example["gt_list"][0].numpy()
    return instance, example

def test_drawing():
    instance, example = get_instance()
    
    draw_from_gt(instance)
    draw_from_raw(gt_to_raw(instance))

loader()
#test_drawing()

# example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
# instance = example["gt_list"][0].numpy()
instance, example = get_instance()

print(instance.shape)
#swap_strokes_left()

INFO:root.hwr_utils.stroke_dataset:Loading data traditional way [PosixPath('online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull/test_online_coords.json')]


/media/data/GitHub/simple_hwr/hwr_utils/testing
../../data/online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull/test_online_coords.json
Loading .npy version


INFO:root.hwr_utils.stroke_dataset:('Original dataloader size', 501)
INFO:root.hwr_utils.stroke_dataset:('Dataloader size', 500)
../../hwr_utils/stroke_dataset.py:236: UserWarning: UNK NUMBER OF SAMPLES!!!
  warnings.warn("UNK NUMBER OF SAMPLES!!!")
INFO:root.hwr_utils.stroke_dataset:('Dataloader size', 500)


(728, 4)


In [69]:
# What if we only target i's?
    # Very short strokes
    # Swap over if the furtherest left stroke of next stroke is further right than the current one

def test_drawing_swaps(gt):
    #print(gt.shape)
    data = draw_from_gt(gt, show=False, right_padding="random", color=(0,0,0), linewidth=1, use_stroke_number=False)[::-1]
    img = Image.fromarray(data, "RGB")
    #img.show()

    # Create an image
    swapped_strokes = Image.new("RGB", (data.shape[:2][::-1]), (255, 255, 255))
    swapped_strokes.paste(img)
    draw = ImageDraw.Draw(swapped_strokes)
    
    gt = gt.copy()
    gt[:,:2] *= 61
    gt_pil = list(gt_to_pil(gt, stroke_number=False))
    new_gt = swap_strokes_left(gt.copy(), stroke_numbers=False, height=61)
    new_gt_pil = list(gt_to_pil(new_gt, stroke_number=False))      
    show=False
    
    for i in range(len(new_gt_pil)):
        if len(gt_pil[i]) != len(new_gt_pil[i]) or not np.allclose(gt_pil[i], new_gt_pil[i]):
            #print(gt_pil[i][:10])
            draw.line(gt_pil[i], fill=(0,0,255), width=2)
            draw.line(new_gt_pil[i], fill=(0,255,0), width=2)
            draw_ellipse(gt_pil[i], draw, color=(0,0,255), width=3)
            draw_ellipse(new_gt_pil[i], draw, color=(0,255,0), width=3)

            
            show=True
            break
    if show:
        Image.fromarray((np.array(swapped_strokes)[::-1]), "RGB").show()

def draw_ellipse(point, draw, color=(0,0,255), width=2):
    line1 = [x-width/2 for x in point[0]]
    line2 = [x+width/2 for x in point[0]]
    point = np.r_[line1, line2].flatten().tolist()
    draw.ellipse(point, fill=color, outline=color)

    #overlay_images(background, instance)

In [70]:
for i in test_dataloader:
    for ii in i["gt_list"]:
        test_drawing_swaps(ii.numpy())
    input()

Process Process-65:
Process Process-66:
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fabafd009d0>
Traceback (most recent call last):
  File "/home/taylor/anaconda3/envs/hwr5/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/home/taylor/anaconda3/envs/hwr5/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/home/taylor/anaconda3/envs/hwr5/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/taylor/anaconda3/envs/hwr5/lib/python3.8/multiprocessing/popen_fork.py", line 47, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/home/taylor/anaconda3/envs/hwr5/lib/python3.8/multiprocessing/popen_fork.py", line 27, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 
Traceback (most recent call last):
Traceback (most recent call last):
 

KeyboardInterrupt: 

In [61]:
gt = np.array(range(36)).reshape(9,4)/61
gt[:,2] = [1,0,0,1,0,1,1,0,0]
gt2=gt.copy()
test_drawing_swaps(gt)


In [55]:
x = gt_to_pil(gt2, stroke_number=False)
print(list(x))

[[(0.0, 0.01639344262295082), (0.06557377049180328, 0.08196721311475409), (0.13114754098360656, 0.14754098360655737)], [(0.19672131147540983, 0.21311475409836064), (0.26229508196721313, 0.2786885245901639)], [(0.32786885245901637, 0.3442622950819672)], [(0.39344262295081966, 0.4098360655737705), (0.45901639344262296, 0.47540983606557374), (0.5245901639344263, 0.5409836065573771)]]
